# commit

> Route handler for committing the document to the context graph

In [ ]:
#| default_exp routes.commit

In [ ]:
#| export
from typing import Dict, Callable, Tuple, Any, Optional
from dataclasses import dataclass

from fasthtml.common import APIRouter, Div, Span, Script

from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, items, gap
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors

from cjm_fasthtml_lucide_icons.factory import lucide_icon

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_plugin_system.core.manager import PluginManager

from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.services.graph import GraphService
from cjm_transcript_review.routes.core import (
    WorkflowStateStore, _load_review_context, _get_review_state
)

# Debug flag
DEBUG_COMMIT_ROUTES = False

## Commit Result

Data structure for commit operation results.

In [ ]:
#| export
@dataclass
class CommitResult:
    """Result of a commit operation."""
    success: bool  # Whether the commit succeeded
    document_id: Optional[str] = None  # Created document node ID
    segment_count: int = 0  # Number of segments committed
    edge_count: int = 0  # Number of edges created
    error: Optional[str] = None  # Error message if failed

## Alert Rendering

Renders success/error alerts with auto-dismiss functionality.

In [ ]:
#| export
COMMIT_ALERT_ID = "commit-alert"

def _render_commit_alert(
    result:CommitResult,  # Commit operation result
    auto_dismiss_ms:int=5000,  # Auto-dismiss after milliseconds (0 = no auto-dismiss)
) -> Div:  # Alert element with optional auto-dismiss script
    """Render a success or error alert for commit result."""
    if result.success:
        icon = lucide_icon("circle-check")
        message = f"Committed {result.segment_count} segments to graph"
        alert_cls = combine_classes(alert, alert_colors.success)
    else:
        icon = lucide_icon("circle-alert")
        message = result.error or "Commit failed"
        alert_cls = combine_classes(alert, alert_colors.error)
    
    content = Div(
        icon,
        Span(message),
        cls=combine_classes(flex_display, items.center, gap(2))
    )
    
    alert_div = Div(
        content,
        id=COMMIT_ALERT_ID,
        cls=combine_classes(alert_cls, m.y(2))
    )
    
    # Auto-dismiss script with fade-out
    if auto_dismiss_ms > 0:
        dismiss_script = Script(f"""
            (function() {{
                const alertEl = document.getElementById('{COMMIT_ALERT_ID}');
                if (alertEl) {{
                    // Start fade after delay
                    setTimeout(function() {{
                        alertEl.style.transition = 'opacity 0.5s ease-out';
                        alertEl.style.opacity = '0';
                        // Remove from DOM after fade
                        setTimeout(function() {{
                            alertEl.remove();
                        }}, 500);
                    }}, {auto_dismiss_ms});
                }}
            }})();
        """)
        return Div(alert_div, dismiss_script)
    
    return alert_div

## Commit Handler

Core handler that performs the commit operation.

In [ ]:
#| export
async def _handle_commit(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str,  # Session identifier string
    graph_service:GraphService,  # Graph service for committing
    document_title:Optional[str]=None,  # Override document title (None = use state)
) -> CommitResult:  # Result of the commit operation
    """Handle committing the document to the context graph."""
    # Load context to get segments and VAD chunks
    ctx = _load_review_context(state_store, workflow_id, session_id)
    
    if DEBUG_COMMIT_ROUTES:
        print(f"[COMMIT] Segments: {len(ctx.segments)}, VAD chunks: {len(ctx.vad_chunks)}")
    
    # Validate alignment
    if len(ctx.segments) != len(ctx.vad_chunks):
        return CommitResult(
            success=False,
            error=f"Segment count ({len(ctx.segments)}) does not match VAD chunk count ({len(ctx.vad_chunks)})"
        )
    
    if len(ctx.segments) == 0:
        return CommitResult(
            success=False,
            error="No segments to commit"
        )
    
    # Check graph service availability
    if not graph_service.is_available():
        return CommitResult(
            success=False,
            error="Graph plugin not loaded"
        )
    
    # Get document title from state if not provided
    if document_title is None:
        review_state = _get_review_state(state_store, workflow_id, session_id)
        document_title = review_state.get("document_title", "Untitled Document")
    
    try:
        # Commit to graph
        result = await graph_service.commit_document_async(
            title=document_title,
            text_segments=ctx.segments,
            vad_chunks=ctx.vad_chunks,
            media_type="audio",
        )
        
        if DEBUG_COMMIT_ROUTES:
            print(f"[COMMIT] Success: doc_id={result['document_id']}, edges={result['edge_count']}")
        
        return CommitResult(
            success=True,
            document_id=result["document_id"],
            segment_count=len(result["segment_ids"]),
            edge_count=result["edge_count"],
        )
    except Exception as e:
        if DEBUG_COMMIT_ROUTES:
            print(f"[COMMIT] Error: {e}")
        return CommitResult(
            success=False,
            error=str(e)
        )

## Router Initialization

Creates commit route and returns the router with route function.

In [ ]:
#| export
def init_commit_router(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    prefix:str,  # Base prefix for commit route
    graph_service:GraphService,  # Graph service for committing
    urls:ReviewUrls,  # URL bundle to populate
    alert_container_id:str="commit-alert-container",  # ID of container for alert OOB swap
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, routes dict)
    """Initialize commit route."""
    router = APIRouter(prefix=prefix)
    routes = {}
    
    @router.post("/commit")
    async def commit(request, sess):
        """Handle commit to context graph."""
        session_id = get_session_id(sess)
        
        if DEBUG_COMMIT_ROUTES:
            print(f"[COMMIT_ROUTE] Commit requested")
        
        # Perform commit
        result = await _handle_commit(
            state_store, workflow_id, session_id, graph_service
        )
        
        # Render alert with OOB swap
        alert_content = _render_commit_alert(result)
        
        return Div(
            alert_content,
            id=alert_container_id,
            hx_swap_oob="innerHTML"
        )
    
    routes["commit"] = commit
    
    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()